## About this document
This document contains the analysis and report for the Assessment of CASA0006-Data Science for Spatial Systems

Student ID: 21203086
Word count: XXX

--------------------------------------------------------------------------------------------------------------------------------

# TITLE XXXXXX

<a id="ID_intr"></a>
| [1.0 Introduction](#ID_intro) | [2.0 Literature review](#ID_lit) | [3.0 Research Question](#ID_RQ) | [4.0 Presentation of Data](#ID_data) | [5.0 Methodology](#ID_methods) | [6.0 Results](#ID_results) | [7.0 Discussion](#ID_disc) | [8.0 Conclusion](#ID_concl) | [Bibliography](#ID_biblio) | [Appendix](#ID_append) |

<a id="ID_intro"></a>
## 1.0 | Introduction 

This paper studies .... 
quick summary
text text

According to the Understanding Inequalities project ''the housing market has a key role in "sorting" poorer households into areas with the worst pollution, schools, crime and employ-ment. ''(Housing | Understanding Inequalities, 2022). This study will therefore investigate if there is a relationship between a variety of indicators for environmental, health and socioeco-nomic features that compose a census tract and house prices in the same area. The case study chosen is California and the baseline assumption is that pollution, unemployment, and poor health statistics will be reflected in the overall house price trend. 

<a id="ID_lit"></a>
## 2.0 | Literature Review

Text text text


<a id="ID_RQ"></a>
## 3.0 | Research Question

R.Q.: Can house prices be predicted from data on environmental, health, and socioeconomic information of a census tract?

    RQ1 | text 
    RQ2 | text?

<a id="ID_data"></a>
## 4.0 | Presentation of Data

The data used for this analysis were obtained from the following sources:
* [CalEnviroScreen 3.0 Results](https://data.ca.gov/dataset/calenviroscreen-3-0-results/resource/89b3f4e9-0bf8-4690-8c6f-715a717f3fae), downloaded 04 April 2021:

   - Who
   [FEDERAL HOUSING FINACE AGENCY](https://www.fhfa.gov/)
   - Where from
   - What
   - Data 
   - Explanation/description

* [House Price Index Census Tracts](https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index-Datasets.aspx), downloaded 04 April 2021:

   - Who
   - Where from
   - What
   - Data 
   - House price index definition

According to the data dictionary provided on the website:
"The FHFA House Price Index (HPI) is a broad measure of the movement of single-family house prices.  The HPI is a weighted, repeat-sales index, meaning that it measures average price changes in repeat sales or refinancings on the same properties. This information is obtained by reviewing repeat mortgage transactions on single-family properties whose mortgages have been purchased or securitized by Fannie Mae or Freddie Mac since January 1975."



For ease of access the data has been saved in this [GitHub repository](https://github.com/DaphneBdn/dsss_repo) 

<a id="ID_methods"></a>
## 5.0 | Methodology

Outline of methodology....text text text

5.1 Libraries

Loading the necessary libraries

In [55]:
# data manipulation and processing
#import os
import math
import numpy as np
import pandas as pd

# analysis
import re #regular expressions

# Packages for visualisation
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns

# reproducability settings
random_state = 42 
np.random.seed(random_state)


5.1 Data Loading 

In [2]:
# Load in the House Price Index data 
#dfprice = pd.read_csv('https://github.com/DaphneBdn/dsss_repo/blob/main/data/raw/HPI_AT_BDL_tract.csv')
dfprice = pd.read_csv('data/raw/HPI_AT_BDL_tract.csv')
dfprice.head(5)

,tract,state_abbr,year,annual_change,hpi,hpi1990,hpi2000
0,1001020100,AL,1998,NaN,100.000000,NaN,100.82
1,1001020100,AL,1999,-5.60,94.400002,NaN,95.18
2,1001020100,AL,2000,5.06,99.180000,NaN,100.00
3,1001020100,AL,2001,7.08,106.210000,NaN,107.08
4,1001020100,AL,2002,7.11,113.760000,NaN,114.70


In [3]:
dfprice.tail(5)

,tract,state_abbr,year,annual_change,hpi,hpi1990,hpi2000
1048570,29007950100,MO,2003,NaN,100.00,NaN,NaN
1048571,29007950100,MO,2004,11.68,111.68,NaN,NaN
1048572,29007950100,MO,2005,-6.56,104.36,NaN,NaN
1048573,29007950100,MO,2006,7.93,112.63,NaN,NaN
1048574,29007950100,MO,2007,-9.61,101.81,NaN,NaN


In [4]:
#check the size of the dataframe
dfprice.shape

(1048575, 7)

In [5]:
# check the datatype of each column 
dfprice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 7 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tract          1048575 non-null  int64  
 1   state_abbr     1048575 non-null  object 
 2   year           1048575 non-null  int64  
 3   annual_change  968146 non-null   float64
 4   hpi            1017771 non-null  float64
 5   hpi1990        542092 non-null   float64
 6   hpi2000        900951 non-null   float64
dtypes: float64(4), int64(2), object(1)
memory usage: 56.0+ MB


After the first inspection of the data the key things to note are: 
* There are 1,048,575 entries in total
* There are multiple NA values: discuss approach, which columns
* years 1998-2021

#### subseting/filtering

* Extract census tracts of California only.
* Keep hpi 
* Year: Ideally keep only 2018, so as to match with the Enviro Dataset.
* If many NAs in 2018: perhaps combine the year before and after? 

In [6]:
dfCAall = dfprice[dfprice['state_abbr']=="CA"]

In [7]:
dfCAall.shape

(264546, 7)

In [8]:
columns = dfCAall.columns
columns

In [8]:
todrop=['annual_change', 'hpi1990','hpi2000']

In [9]:
dfCAall.drop(columns=todrop, inplace=True)
dfCAall

/opt/conda/envs/sds2021/lib/python3.9/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,tract,state_abbr,year,hpi
77893,6001400100,CA,1986,100.00000
77894,6001400100,CA,1987,103.27000
77895,6001400100,CA,1988,129.86000
77896,6001400100,CA,1989,142.83000
77897,6001400100,CA,1990,150.85001
...,...,...,...,...
342434,6115041102,CA,2017,175.52000
342435,6115041102,CA,2018,199.45000
342436,6115041102,CA,2019,211.89999
342437,6115041102,CA,2020,224.97000


In [10]:
# check the number of tracts
dfCAall['tract'].nunique()

7720

In [11]:
dfCA2018= dfCAall[dfCAall['year']==2018]
dfCA2018

,tract,state_abbr,year,hpi
77925,6001400100,CA,2018,548.07001
77961,6001400200,CA,2018,647.09998
78004,6001400300,CA,2018,1491.56010
78047,6001400400,CA,2018,1559.68010
78083,6001400500,CA,2018,789.46997
...,...,...,...,...
342313,6115040901,CA,2018,196.80000
342345,6115041001,CA,2018,249.62000
342376,6115041002,CA,2018,250.35001
342406,6115041101,CA,2018,209.84000


In [ ]:
yearlist = [2018,2019,2020,2021]

for i in yearlist: 
    dftemp = dfCAall[dfCAall['year']==i]
    dftemp.isna().sum().sort_values(ascending=False)
    

In [12]:
# change settings to display all rows
#pd.set_option('display.max_rows', None)

# investigate NA values by printing out the count of NA values per column 
dfCA2018.isna().sum().sort_values(ascending=False)

# change settings back 
#pd.reset_option('all')

hpi           863
tract           0
state_abbr      0
year            0
dtype: int64

In [79]:
max(dfCA2018.isna().sum())

863

In [13]:
dfCA2017= dfCAall[dfCAall['year']==2017]
dfCA2017.isna().sum().sort_values(ascending=False)

hpi           570
tract           0
state_abbr      0
year            0
dtype: int64

In [42]:
dfCA2019= dfCAall[dfCAall['year']==2019]
dfCA2019.isna().sum().sort_values(ascending=False)

hpi           551
tract           0
state_abbr      0
year            0
dtype: int64

In [43]:
dfCA2020= dfCAall[dfCAall['year']==2020]
dfCA2020.isna().sum().sort_values(ascending=False)

hpi           89
tract          0
state_abbr     0
year           0
dtype: int64

In [44]:
dfCA2021= dfCAall[dfCAall['year']==2021]
dfCA2021.isna().sum().sort_values(ascending=False)

hpi           149
tract           0
state_abbr      0
year            0
dtype: int64

In [45]:
dfCA2020[dfCA2020.hpi.isna()]

,tract,state_abbr,year,hpi
82028,6001423602,CA,2020,NaN
87340,6001441927,CA,2020,NaN
99434,6013359104,CA,2020,NaN
112053,6029001105,CA,2020,NaN
112497,6029002806,CA,2020,NaN
...,...,...,...,...
310596,6085511502,CA,2020,NaN
310699,6085511704,CA,2020,NaN
310729,6085511705,CA,2020,NaN
318774,6095251802,CA,2020,NaN


In [48]:
# test if there is information in another year
dfCA2017[dfCA2017['tract']==6085511502]

,tract,state_abbr,year,hpi
310593,6085511502,CA,2017,NaN


In [15]:
dfCA3 = dfCAall[(dfCAall['year']==2017) | (dfCAall['year']==2018) |(dfCAall['year']==2019)]
dfCA3.isna().sum().sort_values(ascending=False)

hpi           1984
tract            0
state_abbr       0
year             0
dtype: int64

####  **envir data**

In [16]:
# Load in the Enviro data 
#dfenv = pd.read_csv('https://github.com/DaphneBdn/dsss_repo/blob/main/data/raw/calenviroscreen-3.0-results-june-2018-update.csv')
dfenv = pd.read_csv('data/raw/calenviroscreen-3.0-results-june-2018-update.csv')
dfenv.head(5)

,Census Tract,Total Population,California County,ZIP,Nearby City \n(to help approximate location only),Longitude,Latitude,CES 3.0 Score,CES 3.0 Percentile,CES 3.0 \nPercentile Range,...,Linguistic Isolation Pctl,Poverty,Poverty Pctl,Unemployment,Unemployment Pctl,Housing Burden,Housing Burden Pctl,Pop. Char.,Pop. Char. Score,Pop. Char. Pctl
0,6019001100,3174,Fresno,93706,Fresno,-119.781696,36.709695,94.09,100.00,95-100% (highest scores),...,77.51,76.3,97.12,17.6,91.72,26.0,79.40,92.12,9.55,99.70
1,6071001600,6133,San Bernardino,91761,Ontario,-117.618013,34.057780,90.68,99.99,95-100% (highest scores),...,96.25,72.5,94.63,12.3,71.82,34.1,93.75,87.44,9.07,98.11
2,6019000200,3167,Fresno,93706,Fresno,-119.805504,36.735491,85.97,99.97,95-100% (highest scores),...,78.39,86.8,99.56,16.1,87.98,40.1,97.85,94.58,9.81,99.99
3,6077000801,6692,San Joaquin,95203,Stockton,-121.314524,37.940517,82.49,99.96,95-100% (highest scores),...,75.14,61.3,85.57,19.6,94.97,21.1,63.54,86.70,8.99,97.72
4,6019001500,2206,Fresno,93725,Fresno,-119.717843,36.681600,82.03,99.95,95-100% (highest scores),...,73.72,66.4,90.23,18.6,93.65,28.1,83.98,80.08,8.30,92.76


In [17]:
# check size of dataframe
dfenv.shape

(8035, 57)

In [30]:
dfenv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8035 entries, 0 to 8034
Data columns (total 57 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Census Tract                                      8035 non-null   int64  
 1   Total Population                                  8035 non-null   int64  
 2   California County                                 8035 non-null   object 
 3   ZIP                                               8035 non-null   int64  
 4   Nearby City 
(to help approximate location only)  8035 non-null   object 
 5   Longitude                                         8035 non-null   float64
 6   Latitude                                          8035 non-null   float64
 7   CES 3.0 Score                                     7929 non-null   float64
 8    CES 3.0 Percentile                               7929 non-null   float64
 9   CES 3.0 
Percentile

In [18]:
dfenv.isna().sum().sort_values(ascending=False)

Linguistic Isolation                                 242
Linguistic Isolation Pctl                            242
Low Birth Weight                                     222
Low Birth Weight Pctl                                222
Housing Burden Pctl                                  157
Housing Burden                                       157
Unemployment Pctl                                    155
Unemployment                                         155
CES 3.0 \nPercentile Range                           106
Pop. Char. Pctl                                      106
 CES 3.0 Percentile                                  106
CES 3.0 Score                                        106
Pop. Char. Score                                     106
Pop. Char.                                           106
Education                                             96
Education Pctl                                        96
Poverty Pctl                                          79
Poverty                        

In [19]:
# check the number of tracts in the combined df
dfenv['Census Tract'].nunique()

8035

NOTE 
According to [United Staes Census Bureau](https://www.census.gov/geographies/reference-files/2010/geo/state-local-geo-guides-2010/california.html), California has **8,057** census tracts

The Enviro dataset lists 8035
The HPI lists 7720 for all years, 7711 for 2018

In [20]:
# extract the column names
cols=dfenv.columns
print(cols)
print()
print(f"The Enviro 3.0 dataset has {len(cols)} columns")

Index(['Census Tract', 'Total Population', 'California County', 'ZIP',
       'Nearby City \n(to help approximate location only)', 'Longitude',
       'Latitude', 'CES 3.0 Score', ' CES 3.0 Percentile',
       'CES 3.0 \nPercentile Range', 'SB 535 Disadvantaged Community', 'Ozone',
       'Ozone Pctl', 'PM2.5', 'PM2.5 Pctl', 'Diesel PM', 'Diesel PM Pctl',
       'Drinking Water', 'Drinking Water Pctl', 'Pesticides',
       'Pesticides Pctl', 'Tox. Release', 'Tox. Release Pctl', 'Traffic',
       'Traffic Pctl', 'Cleanup Sites', 'Cleanup Sites Pctl',
       'Groundwater Threats', 'Groundwater Threats Pctl', 'Haz. Waste',
       'Haz. Waste Pctl', 'Imp. Water Bodies', 'Imp. Water Bodies Pctl',
       'Solid Waste', 'Solid Waste Pctl', 'Pollution Burden',
       'Pollution Burden Score', 'Pollution Burden Pctl', 'Asthma',
       'Asthma Pctl', 'Low Birth Weight', 'Low Birth Weight Pctl',
       'Cardiovascular Disease', 'Cardiovascular Disease Pctl', 'Education',
       'Education Pctl', 

### newer version

In [50]:
#env21= pd.read_excel('https://github.com/DaphneBdn/dsss_repo/blob/main/data/raw/calenviroscreen40resultsdatadictionary_F_2021.xlsx', sheet_name="CES4.0FINAL_results")

env21= pd.read_excel('data/raw/calenviroscreen40resultsdatadictionary_F_2021.xlsx', sheet_name="CES4.0FINAL_results")
env21.head()

,Census Tract,Total Population,California County,ZIP,Approximate Location,Longitude,Latitude,CES 4.0 Score,CES 4.0 Percentile,CES 4.0 Percentile Range,...,Linguistic Isolation Pctl,Poverty,Poverty Pctl,Unemployment,Unemployment Pctl,Housing Burden,Housing Burden Pctl,Pop. Char.,Pop. Char. Score,Pop. Char. Pctl
0,6019001100,2780,Fresno,93706,Fresno,-119.781696,36.709695,93.183570,100.000000,95-100% (highest scores),...,79.374746,76.0,98.919598,12.8,93.831338,30.3,91.039290,93.155109,9.663213,99.722642
1,6077000700,4680,San Joaquin,95206,Stockton,-121.287873,37.943173,86.653790,99.987393,95-100% (highest scores),...,95.533902,73.2,98.391960,19.8,99.206143,31.2,92.281369,93.165408,9.664281,99.735250
2,6037204920,2751,Los Angeles,90023,Los Angeles,-118.197497,34.017500,82.393909,99.974786,95-100% (highest scores),...,81.553661,62.6,93.391960,6.4,61.530453,20.3,63.967047,83.751814,8.687785,95.789208
3,6019000700,3664,Fresno,93706,Fresno,-119.827707,36.734535,81.327940,99.962179,95-100% (highest scores),...,78.711598,65.7,95.351759,15.7,97.345133,35.4,96.413181,94.641227,9.817371,99.886536
4,6019000200,2689,Fresno,93706,Fresno,-119.805504,36.735491,80.745476,99.949571,95-100% (highest scores),...,86.561104,72.7,98.304020,13.7,95.288912,32.7,94.157161,95.398873,9.895964,99.949571


In [51]:
env21.shape

(8035, 58)

In [53]:
env21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8035 entries, 0 to 8034
Data columns (total 58 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Census Tract                 8035 non-null   int64  
 1   Total Population             8035 non-null   int64  
 2   California County            8035 non-null   object 
 3   ZIP                          8035 non-null   int64  
 4   Approximate Location         8035 non-null   object 
 5   Longitude                    8035 non-null   float64
 6   Latitude                     8035 non-null   float64
 7   CES 4.0 Score                7932 non-null   float64
 8    CES 4.0 Percentile          7932 non-null   float64
 9   CES 4.0 Percentile Range     7932 non-null   object 
 10  Ozone                        8035 non-null   float64
 11  Ozone Pctl                   8035 non-null   float64
 12  PM2.5                        8035 non-null   float64
 13  PM2.5 Pctl        

In [54]:
# extract the column names
cols=env21.columns
print(cols)
print()
print(f"The Enviro 3.0 dataset has {len(cols)} columns")

Index(['Census Tract', 'Total Population', 'California County', 'ZIP',
       'Approximate Location', 'Longitude', 'Latitude', 'CES 4.0 Score',
       ' CES 4.0 Percentile', 'CES 4.0 Percentile Range', 'Ozone',
       'Ozone Pctl', 'PM2.5', 'PM2.5 Pctl', 'Diesel PM', 'Diesel PM Pctl',
       'Drinking Water', 'Drinking Water Pctl', 'Lead', 'Lead Pctl',
       'Pesticides', 'Pesticides Pctl', 'Tox. Release', 'Tox. Release Pctl',
       'Traffic', 'Traffic Pctl', 'Cleanup Sites', 'Cleanup Sites Pctl',
       'Groundwater Threats', 'Groundwater Threats Pctl', 'Haz. Waste',
       'Haz. Waste Pctl', 'Imp. Water Bodies', 'Imp. Water Bodies Pctl',
       'Solid Waste', 'Solid Waste Pctl', 'Pollution Burden',
       'Pollution Burden Score', 'Pollution Burden Pctl', 'Asthma',
       'Asthma Pctl', 'Low Birth Weight', 'Low Birth Weight Pctl',
       'Cardiovascular Disease', 'Cardiovascular Disease Pctl', 'Education',
       'Education Pctl', 'Linguistic Isolation', 'Linguistic Isolation Pctl'

In [56]:
env21.isna().sum().sort_values(ascending=False)

Unemployment Pctl              335
Unemployment                   335
Linguistic Isolation           320
Linguistic Isolation Pctl      320
Low Birth Weight Pctl          227
Low Birth Weight               227
Housing Burden Pctl            145
Housing Burden                 145
Pop. Char. Pctl                103
Education Pctl                 103
Education                      103
CES 4.0 Percentile Range       103
 CES 4.0 Percentile            103
CES 4.0 Score                  103
Pop. Char.                     103
Pop. Char. Score               103
Lead                            96
Lead Pctl                       96
Poverty Pctl                    75
Poverty                         75
Traffic Pctl                    35
Traffic                         35
Drinking Water                  28
Drinking Water Pctl             28
Cardiovascular Disease          11
Cardiovascular Disease Pctl     11
Asthma Pctl                     11
Asthma                          11
Solid Waste         

In [63]:
df = env21[env21.columns.drop(list(env21.filter(regex='Pctl')))]

In [65]:
df = df[df.columns.drop(list(df.filter(regex='CES')))]
df.head()

,Census Tract,Total Population,California County,ZIP,Approximate Location,Longitude,Latitude,Ozone,PM2.5,Diesel PM,...,Asthma,Low Birth Weight,Cardiovascular Disease,Education,Linguistic Isolation,Poverty,Unemployment,Housing Burden,Pop. Char.,Pop. Char. Score
0,6019001100,2780,Fresno,93706,Fresno,-119.781696,36.709695,0.060311,13.906348,1.122712,...,129.54,7.80,21.47,44.5,16.0,76.0,12.8,30.3,93.155109,9.663213
1,6077000700,4680,San Joaquin,95206,Stockton,-121.287873,37.943173,0.045884,11.884085,0.538105,...,105.88,6.88,20.26,46.4,29.7,73.2,19.8,31.2,93.165408,9.664281
2,6037204920,2751,Los Angeles,90023,Los Angeles,-118.197497,34.017500,0.047920,12.251640,0.780833,...,76.10,7.11,20.87,52.2,17.1,62.6,6.4,20.3,83.751814,8.687785
3,6019000700,3664,Fresno,93706,Fresno,-119.827707,36.734535,0.060311,13.520939,0.173815,...,139.45,10.65,22.68,41.4,15.7,65.7,15.7,35.4,94.641227,9.817371
4,6019000200,2689,Fresno,93706,Fresno,-119.805504,36.735491,0.060311,13.818959,1.389658,...,139.08,10.25,22.64,43.6,20.0,72.7,13.7,32.7,95.398873,9.895964


In [68]:
df.dropna(axis=0, how='any', inplace=True)

In [69]:
df.isna().sum().sort_values(ascending=False)

Census Tract              0
Total Population          0
Pop. Char.                0
Housing Burden            0
Unemployment              0
Poverty                   0
Linguistic Isolation      0
Education                 0
Cardiovascular Disease    0
Low Birth Weight          0
Asthma                    0
Pollution Burden Score    0
Pollution Burden          0
Solid Waste               0
Imp. Water Bodies         0
Haz. Waste                0
Groundwater Threats       0
Cleanup Sites             0
Traffic                   0
Tox. Release              0
Pesticides                0
Lead                      0
Drinking Water            0
Diesel PM                 0
PM2.5                     0
Ozone                     0
Latitude                  0
Longitude                 0
Approximate Location      0
ZIP                       0
California County         0
Pop. Char. Score          0
dtype: int64

In [70]:
df.shape

(7355, 32)

In [ ]:
# merge the census data with the previously joined dataframe
London_profiles = pd.merge( joined_wards, ward_data ,left_on = "GSS_CODE", right_on = "Ward code", how = "left")

<a id="ID_results"></a>
## 6.0 | Results

<a id="ID_disc"></a>
## 7.0 | Discussion 

<a id="ID_conc"></a>
## 8.0 | Conclusion

<a id="ID_biblio"></a>
## Bibliography

<a id="ID_append"></a>
## Appendix